In [54]:
import numpy as np

import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### 데이터 작업하기

In [29]:
train_data = datasets.FashionMNIST(
                                    root = 'data',
                                    train = True,
                                    download = True,
                                    transform = ToTensor(),
)

test_data = datasets.FashionMNIST(
                                    root = 'data',
                                    train = False,
                                    download = True,
                                    transform = ToTensor(),
)

batch_size = 64

# 데이터로더 생성
train_dataloader = DataLoader(train_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} ({y.dtype})')
    
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) (torch.int64)


### 모델 만들기

In [30]:
device = 'cpu'
print(f'Using {device}: device')

# model define
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() # nn.Module 클래스를 상속 클래스로 가져온다.
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
                                                nn.Linear(28  * 28, 512),
                                                nn.ReLU(),
                                                nn.Linear(512, 512),
                                                nn.ReLU(),
                                                nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        
        logits = self.linear_relu_stack(x)
        
        return logits
    

model = NeuralNetwork().to(device)
print(model)

Using cpu: device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 모델 매개변수 최적화하기

In [31]:
# 손실함수
loss_fn = nn.CrossEntropyLoss()

# 최적화 함수
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [42]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y) #  nn.CrossEntropyLoss()(pred, y)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')
            
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    model.eval()
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches # 배치 단위로 평균 값을 구하는 것
    correct /= size
    
    print(f'Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f}\n')

In [43]:
epochs = 5

for t in range(epochs):
    print(f'Epoch {t+1}\n--------------------------')
    
    train(train_dataloader, model, loss_fn, optimizer)
    test(train_dataloader, model, loss_fn)
    
print('done')

Epoch 1
--------------------------
loss: 1.921444 [    0/60000]
loss: 1.900442 [ 6400/60000]
loss: 1.776031 [12800/60000]
loss: 1.824465 [19200/60000]
loss: 1.727718 [25600/60000]
loss: 1.663459 [32000/60000]
loss: 1.688415 [38400/60000]
loss: 1.581881 [44800/60000]
loss: 1.609367 [51200/60000]
loss: 1.508349 [57600/60000]
Test Error: 
 Accuracy: 63.6%, Avg loss: 1.515429

Epoch 2
--------------------------
loss: 1.581369 [    0/60000]
loss: 1.555866 [ 6400/60000]
loss: 1.397086 [12800/60000]
loss: 1.476476 [19200/60000]
loss: 1.360474 [25600/60000]
loss: 1.344947 [32000/60000]
loss: 1.366002 [38400/60000]
loss: 1.280496 [44800/60000]
loss: 1.321052 [51200/60000]
loss: 1.225077 [57600/60000]
Test Error: 
 Accuracy: 65.0%, Avg loss: 1.239481

Epoch 3
--------------------------
loss: 1.319220 [    0/60000]
loss: 1.310707 [ 6400/60000]
loss: 1.138401 [12800/60000]
loss: 1.250935 [19200/60000]
loss: 1.121141 [25600/60000]
loss: 1.142994 [32000/60000]
loss: 1.173743 [38400/60000]
loss: 1.09

### 모델 저장 및 불러오기

In [44]:
torch.save(model.state_dict(), './data/pytorch_basic_model.pth')
print('Saved PyTorch Model State to pytorch_basic_model.pth')

Saved PyTorch Model State to pytorch_basic_model.pth


In [47]:
model = NeuralNetwork()
model.load_state_dict(torch.load('./data/pytorch_basic_model.pth'))

<All keys matched successfully>

In [48]:
classes = [
            "T-shirt/top",
            "Trouser",
            "Pullover",
            "Dress",
            "Coat",
            "Sandal",
            "Shirt",
            "Sneaker",
            "Bag",
            "Ankle boot",
]

model.eval()

x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"



--------------------------------


# 텐서 (Tensor)

### 텐서 초기화

In [52]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)

print(data, type(data))
print(x_data, x_data.shape, type(x_data))

[[1, 2], [3, 4]] <class 'list'>
tensor([[1, 2],
        [3, 4]]) torch.Size([2, 2]) <class 'torch.Tensor'>


torch.tensor 메서드를 통해, 리스트 타입의 데이터를 텐서 타입으로 변경한다. 텐서의 경우 형태(shape)을 가진다.

In [59]:
data = [[1, 2], [3, 4]]
np_array = np.array(data)

x_np_1 = torch.tensor(np_array)
x_np_2 = torch.from_numpy(np_array)

print(data, type(data))
print(np_array, type(np_array))
print(x_np_1, x_np_1.shape, type(x_np_1))
print(x_np_2, x_np_2.shape, type(x_np_2))

[[1, 2], [3, 4]] <class 'list'>
[[1 2]
 [3 4]] <class 'numpy.ndarray'>
tensor([[1, 2],
        [3, 4]], dtype=torch.int32) torch.Size([2, 2]) <class 'torch.Tensor'>
tensor([[1, 2],
        [3, 4]], dtype=torch.int32) torch.Size([2, 2]) <class 'torch.Tensor'>


numpy array 데이트를 텐서로 변경할 경우, 리스트를 변경하던 것과 같이 torch.tensor 메서드로도 변경이 가능하고

torch.from_numpy 메서드를 통해서도 변경이 가능하다.

In [74]:
x_ones = torch.ones_like(x_data)

print(f'Org Tensor:\n {x_data}')
print(f'Ones Tensor:\n {x_ones}')

x_rand_float = torch.rand_like(x_data, dtype = torch.float) # 텐서를 구성하는 데이터의 타입을 변경할 경우, 명시해주어야 한다.

print(f'Random Tensor:\n {x_rand_float}')

Org Tensor:
 tensor([[1, 2],
        [3, 4]])
Ones Tensor:
 tensor([[1, 1],
        [1, 1]])
Random Tensor:
 tensor([[0.2364, 0.9786],
        [0.0341, 0.1483]])


In [76]:
shape = (2, 3, 1)

rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f'Random Tensor: \n {rand_tensor}\n')
print(f'Ones Tensor: \n {ones_tensor}\n')
print(f'Zeros Tensor: \n {zeros_tensor}')

Random Tensor: 
 tensor([[[0.6306],
         [0.7636],
         [0.5492]],

        [[0.5855],
         [0.1014],
         [0.1427]]])

Ones Tensor: 
 tensor([[[1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.]]])

Zeros Tensor: 
 tensor([[[0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.]]])


In [85]:
shape = (2, 3, )

rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f'Random Tensor: \n {rand_tensor}\n')
print(f'Ones Tensor: \n {ones_tensor}\n')
print(f'Zeros Tensor: \n {zeros_tensor}')

Random Tensor: 
 tensor([[0.6434, 0.6479, 0.2607],
        [0.0702, 0.3801, 0.1208]])

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]])

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


### 텐서의 속성

In [92]:
tensor = torch.rand(3, 4, 2)

print(f'Shape of tensor: {tensor.shape}')
print(f'Datatype of tensor: {tensor.dtype}')
print(f'Device tensor is stored on: {tensor.device}')

Shape of tensor: torch.Size([3, 4, 2])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


### 텐서의 연산

In [94]:
# GPU가 존재하면 텐서를 이동한다.
if torch.cuda.is_available():
    tensor = tensor.to('cuda')

In [96]:
tensor = torch.ones(4, 4)

print(tensor)
print(f'First row: {tensor[0]}')
print(f'First column: {tensor[:, 0]}')
print(f'Last column: {tensor[:, -1]}')

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])


In [97]:
tensor[:, 1] = 0 # 1열 값을 전부 0으로 변경

print(tensor)
print(f'First row: {tensor[0]}')
print(f'First column: {tensor[:, 0]}')
print(f'Last column: {tensor[:, -1]}')

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
First row: tensor([1., 0., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])


In [114]:
print(torch.cat([tensor, tensor], dim = 0).shape)
print(torch.cat([tensor, tensor], dim = 0))

print(torch.cat([tensor, tensor], dim = 1).shape)
print(torch.cat([tensor, tensor], dim = 1))

torch.Size([8, 4])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
torch.Size([4, 8])
tensor([[1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1.]])


In [124]:
# 지정해준 차원(dim 값)에 데이터를 stack 하는 것

print(torch.stack([tensor, tensor], dim = 1).shape)
print(torch.stack([tensor, tensor], dim = 1))

torch.Size([4, 2, 4])
tensor([[[1., 0., 1., 1.],
         [1., 0., 1., 1.]],

        [[1., 0., 1., 1.],
         [1., 0., 1., 1.]],

        [[1., 0., 1., 1.],
         [1., 0., 1., 1.]],

        [[1., 0., 1., 1.],
         [1., 0., 1., 1.]]])


In [130]:
# 텐서 간의 행렬 곱
# @ = matmul 같은 기능

y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out = y3)

tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])

In [135]:
# 요소별 곱

z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out = z3)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])